In [2]:
from pathlib import Path
from decode_fish.funcs.utils import *
import glob

def make_sweep_str(pfile, pars, group, resume=False, init=None, gpu='gpu_rtx', runfile='train.py', n=1):
    
    base_string = f'bsub -gpu "num=1" -n {n} -q {gpu} -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/{runfile}'   
    
    base_str = base_string + f' +experiment={pfile}'
    name = ''
    
    for k in pars:
        if k!= 'run_name':
            base_str += ' ' + k + f'={pars[k]}'
            name += k.split('.')[-1]+f':{pars[k]}x'
            name = name.replace('[', '')
            name = name.replace(']', '')
            name = name.replace(',', 'x')
    name = name[:-1]
    
    if 'run_name' in pars:
        name=pars['run_name']
    
    if resume:
        base_str += ' training.resume=True '
    if init is not None:
        base_str += f' data_path.model_init={init} '
    
    return base_str + f' run_name={name} output.group={group}' + '\n'
    

def make_named_str(pfile, name, group):
    
    base_str = base_string + f' +experiment={pfile}'
    
    return base_str + f' run_name={name} output.group={group}' + '\n'

In [179]:
# cfg.training.mic.par_grads.channel_facs = False
# cfg.training.mic.par_grads.channel_shifts = False
# cfg.training.mic.par_grads.theta_par = False
# cfg.training.mic.par_grads.psf_vol = False

In [186]:
group = 'sweep_mop_20'
pfile = 'MERFISH_mop_20'

variable_col = param_iter()
variable_col.add('data_path.image_path', '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_images150.tif', '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_images151.tif', '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_images47.tif')
par_prod = variable_col.param_product()

In [193]:
group = 'sweep_mop_27'
pfile = 'MERFISH_mop_27'

variable_col = param_iter()
variable_col.add('training.net.opt.lr', 2e-4)
variable_col.add('genm.pos_noise.pos_noise_xy', 0.3)
variable_col.add('genm.intensity_dist.int_loc', 1., 2., 3.)
# variable_col.add('genm.microscope.scale', 500, 600, 700, 800, 1000)
# variable_col.add('data_path.image_path', '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_scaled_images47.tif', 
#                                         '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_scaled_images150.tif',
#                                         '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_scaled_images151.tif')
par_prod = variable_col.param_product()

In [68]:
group = 'sim_int_2'
pfile = 'sim_int_2'

variable_col = param_iter()
variable_col.add('training.mic.par_grads.channel_facs', False)
# variable_col.add('training.mic.par_grads.channel_shifts', False)
# variable_col.add('training.mic.par_grads.psf_vol', False)
par_prod = variable_col.param_product()

In [194]:
# Path.mkdir(model_path/group, exist_ok=True)

with open('run_slurm.sh', 'w') as f:
    for pars in par_prod:  
        exec_str = make_sweep_str(pfile, pars, group, resume=False, gpu='gpu_rtx') #, 
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_mop_27 training.net.opt.lr=0.0002 genm.pos_noise.pos_noise_xy=0.3 genm.intensity_dist.int_loc=1.0 run_name=lr:0.0002xpos_noise_xy:0.3xint_loc:1.0 output.group=sweep_mop_27

bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_mop_27 training.net.opt.lr=0.0002 genm.pos_noise.pos_noise_xy=0.3 genm.intensity_dist.int_loc=2.0 run_name=lr:0.0002xpos_noise_xy:0.3xint_loc:2.0 output.group=sweep_mop_27

bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python 

In [153]:
bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_mop_25 data_path.image_path=/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_scaled_images47.tif run_name=images47.tif output.group=sweep_mop_25

bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_mop_25 data_path.image_path=/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_scaled_images150.tif run_name=images150.tif output.group=sweep_mop_25

bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_mop_25 data_path.image_path=/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_scaled_images151.tif run_name=images151.tif output.group=sweep_mop_25

SyntaxError: invalid syntax (4178001711.py, line 1)

In [170]:
group = 'MERFISH_sweep_gen_5'
pfile = 'MERFISH_sweep_gen_5'

variable_col = param_iter()
# variable_col.add('training.schedule', 'null')
# variable_col.add('training.num_iters', 9950)

# variable_col.add('genm.microscope.norm', 'none', 'escort')
# variable_col.add('training.mic.par_grads.z_facs', True)
# variable_col.add('training.mic.mean_diff', True)
variable_col.add('sim.bg_estimation.shuffle_ch', True)
variable_col.add('genm.PSF.psf_extent_zyx',  9)

par_prod = variable_col.param_zip()

In [171]:
# Path.mkdir(model_path/group, exist_ok=True)

with open('run_slurm.sh', 'w') as f:
    for pars in par_prod:  
        exec_str = make_sweep_str(pfile, pars, group, resume=False, gpu='gpu_rtx') #, 
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_sweep_gen_5 sim.bg_estimation.shuffle_ch=True genm.PSF.psf_extent_zyx=9 run_name=shuffle_ch:Truexpsf_extent_zyx:9 output.group=MERFISH_sweep_gen_5



In [121]:
group = 'gensweep_mop_1'
pfile = 'gensweep_mop_1'

variable_col = param_iter()
# variable_col.add('training.mic.par_grads.psf_vol', False)
# variable_col.add('training.mic.par_grads.channel_shifts', False)
# variable_col.add('training.mic.par_grads.color_shifts', False)
variable_col.add('genm.microscope.norm', 'none')
variable_col.add('training.mic.par_grads.z_facs', True)
variable_col.add('training.mic.mean_diff', True)
variable_col.add('training.mic.opt.lr', 0.0005, 0.001)
variable_col.add('genm.microscope.scale', 400)

par_prod = variable_col.param_product()

In [348]:
bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_sweep_gen_3 data_path.psf_path=/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/figures/MFsf_psf1.tif genm.microscope.norm=none run_name=nPSFxnorm:none output.group=MERFISH_sweep_gen_3
        
bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_sweep_gen_3 data_path.psf_path=/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/figures/MFsf_psf1.tif genm.microscope.norm=escort run_name=nPSFxnorm:escort output.group=MERFISH_sweep_gen_3

SyntaxError: invalid syntax (3207431614.py, line 1)